In [61]:
%reload_ext autoreload
%autoreload 2

import optuna

In [62]:
!pwd

'pwd' is not recognized as an internal or external command,
operable program or batch file.


##### Hyperparameter Tuning with Accuracy Pruning


In [63]:
import models_hyper_optim_optuna
import models
import neural_net_model_train


#set folders where models will be saved
def setFolders(evaluation_function, use_gru_instead_of_lstm):    
    if use_gru_instead_of_lstm:
        gru_path_root_model = "models\cnn\gru\\" + evaluation_function
        models_hyper_optim_optuna.createFolder(gru_path_root_model) 
        
        return gru_path_root_model +  "\cnn_bi_gru_"
    else:
        lstm_path_root_model = "models\cnn\lstm\\" + evaluation_function
        models_hyper_optim_optuna.createFolder(lstm_path_root_model) 
        
        return lstm_path_root_model + "\cnn_bi_lstm_"
    

In [64]:
channels = 1
batch_size= 32

args_model = {
    'number_of_classes':models_hyper_optim_optuna.number_of_classes, 
    'input_shape' : (batch_size, channels, models_hyper_optim_optuna.input_size),
    'use_gru_instead_of_lstm':False
    }

args_trial = {
    'max_epochs' : 7,
    'no_trials': 7, 
    'evaluation_function':'accuracy', 
    'USE_AUTOMATIC_MIXED_PRECISION':True
}



#set models that will be used for tunning
cnn_birnn_model = models.CnnBirnnModel(args_model)
models_trainning = neural_net_model_train.ModelsTrainning()

### Accuracy Tunning

In [65]:

from time import time


args_trial['evaluation_function']='accuracy'
path_models_id = setFolders(args_trial['evaluation_function'], args_model['use_gru_instead_of_lstm'])

start = time()


study = optuna.create_study(direction = "maximize", sampler = optuna.samplers.TPESampler(),
                            pruner=optuna.pruners.HyperbandPruner(min_resource=3))

models_hyper_optim_optuna.start_optimize_objective(study, args_trial,
                                                   path_models_id,
                                                   cnn_birnn_model,
                                                   models_trainning)

end = time()


[I 2022-09-27 16:42:21,812] A new study created in memory with name: no-name-c3f73311-0f63-49d0-a287-21c6dc2b68b1


ModelsInterface empty_models


c:\Users\androgo\anaconda3\envs\pytorch_env\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning:

Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.



NetModel(
  (cv1): Conv1d(1, 47, kernel_size=(3,), stride=(7,), bias=False)
  (avgp1): AvgPool1d(kernel_size=(3,), stride=(7,), padding=(0,))
  (bn1): BatchNorm1d(47, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel1): LeakyReLU(negative_slope=0.01)
  (cv2): Conv1d(47, 35, kernel_size=(1,), stride=(6,), bias=False)
  (avgp2): AvgPool1d(kernel_size=(1,), stride=(6,), padding=(0,))
  (bn2): BatchNorm1d(35, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel2): LeakyReLU(negative_slope=0.01)
  (cv3): Conv1d(35, 210, kernel_size=(1,), stride=(6,), bias=False)
  (avgp3): AvgPool1d(kernel_size=(1,), stride=(6,), padding=(0,))
  (bn3): BatchNorm1d(210, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel3): LeakyReLU(negative_slope=0.01)
  (bi_lstm): LSTM(210, 420, num_layers=2, batch_first=True, dropout=0.17971826149649728, bidirectional=True)
  (fc_output): Linear(in_features=840, out_features=32, bias=True)
)


100%|██████████| 46/46 [00:01<00:00, 23.44it/s]


...

100%|██████████| 46/46 [00:02<00:00, 18.18it/s]
[I 2022-09-27 16:43:16,130] Trial 4 pruned. 


NetModel(
  (cv1): Conv1d(1, 244, kernel_size=(3,), stride=(1,), bias=False)
  (avgp1): AvgPool1d(kernel_size=(3,), stride=(1,), padding=(0,))
  (bn1): BatchNorm1d(244, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel1): LeakyReLU(negative_slope=0.01)
  (cv2): Conv1d(244, 237, kernel_size=(3,), stride=(7,), bias=False)
  (avgp2): AvgPool1d(kernel_size=(3,), stride=(7,), padding=(0,))
  (bn2): BatchNorm1d(237, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel2): LeakyReLU(negative_slope=0.01)
  (cv3): Conv1d(237, 74, kernel_size=(1,), stride=(6,), bias=False)
  (avgp3): AvgPool1d(kernel_size=(1,), stride=(6,), padding=(0,))
  (bn3): BatchNorm1d(74, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel3): LeakyReLU(negative_slope=0.01)
  (cv4): Conv1d(74, 138, kernel_size=(1,), stride=(2,), bias=False)
  (avgp4): AvgPool1d(kernel_size=(1,), stride=(2,), padding=(0,))
  (bn4): BatchNorm1d(138, eps=1e-05, momentum=0.1, affine=T

100%|██████████| 46/46 [00:01<00:00, 26.30it/s]
[I 2022-09-27 16:43:23,070] Trial 5 pruned. 


NetModel(
  (cv1): Conv1d(1, 256, kernel_size=(3,), stride=(4,), bias=False)
  (avgp1): AvgPool1d(kernel_size=(3,), stride=(4,), padding=(0,))
  (bn1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel1): LeakyReLU(negative_slope=0.01)
  (cv2): Conv1d(256, 249, kernel_size=(3,), stride=(3,), bias=False)
  (avgp2): AvgPool1d(kernel_size=(1,), stride=(3,), padding=(0,))
  (bn2): BatchNorm1d(249, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel2): LeakyReLU(negative_slope=0.01)
  (cv3): Conv1d(249, 214, kernel_size=(1,), stride=(1,), bias=False)
  (avgp3): AvgPool1d(kernel_size=(1,), stride=(1,), padding=(0,))
  (bn3): BatchNorm1d(214, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel3): LeakyReLU(negative_slope=0.01)
  (bi_lstm): LSTM(214, 428, num_layers=2, batch_first=True, dropout=0.22469552476552002, bidirectional=True)
  (fc_output): Linear(in_features=856, out_features=32, bias=True)
)


100%|██████████| 46/46 [00:01<00:00, 24.42it/s]
[I 2022-09-27 16:43:31,651] Trial 6 pruned. 


In [66]:
trials_finished = study.get_trials()
print('number of trials finished: ', len(trials_finished),'\n')

print('number of the best trial: ',study.best_trial.number)

best_trial = study.best_trial
for key, value in best_trial.params.items():
    print("{}: {}".format(key, value))

number of trials finished:  7 

number of the best trial:  2
n_layers: 2
n_units_l0: 41
no_strides_l0: 4
n_units_l1: 116
no_strides_l1: 6
dropout_l1: 0.32481210324255694
learning_rate: 0.004358658107438173
optimizer: SGD


### Tuning Visualization

In [67]:
import plotly
optuna.visualization.plot_intermediate_values(study)

In [68]:
optuna.visualization.plot_optimization_history(study)

In [69]:
optuna.visualization.plot_parallel_coordinate(study)

[W 2022-09-27 16:43:37,304] Your study has only completed trials with missing parameters.


In [70]:
optuna.visualization.plot_param_importances(study)

In [71]:
optuna.visualization.plot_slice(study)


### F1 score Tunning

In [72]:

args_trial['evaluation_function']='f1_score'
path_models_id = setFolders(args_trial['evaluation_function'], args_model['use_gru_instead_of_lstm'])



study = optuna.create_study(direction = "maximize", sampler = optuna.samplers.TPESampler(),
                            pruner=optuna.pruners.HyperbandPruner(min_resource=3))

models_hyper_optim_optuna.start_optimize_objective(study, args_trial,
                                                   path_models_id,
                                                   cnn_birnn_model, 
                                                   models_trainning)


[I 2022-09-27 16:43:41,901] A new study created in memory with name: no-name-1329917a-e6ea-4dc2-a5f8-19b92b81a67a


ModelsInterface empty_models


c:\Users\androgo\anaconda3\envs\pytorch_env\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning:

Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.



NetModel(
  (cv1): Conv1d(1, 223, kernel_size=(3,), stride=(7,), bias=False)
  (avgp1): AvgPool1d(kernel_size=(3,), stride=(7,), padding=(0,))
  (bn1): BatchNorm1d(223, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel1): LeakyReLU(negative_slope=0.01)
  (cv2): Conv1d(223, 58, kernel_size=(1,), stride=(6,), bias=False)
  (avgp2): AvgPool1d(kernel_size=(1,), stride=(6,), padding=(0,))
  (bn2): BatchNorm1d(58, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel2): LeakyReLU(negative_slope=0.01)
  (cv3): Conv1d(58, 135, kernel_size=(1,), stride=(6,), bias=False)
  (avgp3): AvgPool1d(kernel_size=(1,), stride=(6,), padding=(0,))
  (bn3): BatchNorm1d(135, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel3): LeakyReLU(negative_slope=0.01)
  (cv4): Conv1d(135, 145, kernel_size=(1,), stride=(3,), bias=False)
  (avgp4): AvgPool1d(kernel_size=(1,), stride=(3,), padding=(0,))
  (bn4): BatchNorm1d(145, eps=1e-05, momentum=0.1, affine=T

100%|██████████| 46/46 [00:01<00:00, 37.87it/s]


...

100%|██████████| 46/46 [00:01<00:00, 42.82it/s]



Train set: Average loss: 0.0688, Average accuracy: 2.7136%,
 	 Average f1_score: 0.0650, Average Area Under ROC: 0.4988 



100%|██████████| 46/46 [00:00<00:00, 75.39it/s]
[I 2022-09-27 16:44:40,659] Trial 5 finished with value: 0.04927607625722885 and parameters: {'n_layers': 3, 'n_units_l0': 159, 'no_strides_l0': 2, 'n_units_l1': 44, 'no_strides_l1': 3, 'n_units_l2': 126, 'no_strides_l2': 5, 'dropout_l2': 0.14611960955644387, 'learning_rate': 0.06215186370655115, 'optimizer': 'SGD'}. Best is trial 1 with value: 0.0771181583404541.



Test set: Average loss: 2.7135, Average accuracy: 0.0530%,
 	  Average f1_score: 0.0493, Average Area Under ROC: 0.4993 

NetModel(
  (cv1): Conv1d(1, 190, kernel_size=(3,), stride=(3,), bias=False)
  (avgp1): AvgPool1d(kernel_size=(3,), stride=(3,), padding=(0,))
  (bn1): BatchNorm1d(190, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel1): LeakyReLU(negative_slope=0.01)
  (cv2): Conv1d(190, 50, kernel_size=(3,), stride=(7,), bias=False)
  (avgp2): AvgPool1d(kernel_size=(1,), stride=(7,), padding=(0,))
  (bn2): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel2): LeakyReLU(negative_slope=0.01)
  (cv3): Conv1d(50, 71, kernel_size=(1,), stride=(3,), bias=False)
  (avgp3): AvgPool1d(kernel_size=(1,), stride=(3,), padding=(0,))
  (bn3): BatchNorm1d(71, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel3): LeakyReLU(negative_slope=0.01)
  (cv4): Conv1d(71, 237, kernel_size=(1,), stride=(7,), bias=False)
  (av

100%|██████████| 46/46 [00:02<00:00, 19.94it/s]
[I 2022-09-27 16:44:50,079] Trial 6 pruned. 


In [73]:
trials_finished = study.get_trials()
print('number of trials finished: ', len(trials_finished),'\n')

print('number of the best trial: ',study.best_trial.number)

best_trial = study.best_trial
for key, value in best_trial.params.items():
    print("{}: {}".format(key, value))

number of trials finished:  7 

number of the best trial:  1
n_layers: 2
n_units_l0: 79
no_strides_l0: 6
n_units_l1: 41
no_strides_l1: 1
dropout_l1: 0.40615172184301174
learning_rate: 0.019420948883026593
optimizer: RMSprop


### Tuning Visualization

In [74]:
import plotly
optuna.visualization.plot_intermediate_values(study)

In [75]:
optuna.visualization.plot_optimization_history(study)

In [76]:
optuna.visualization.plot_parallel_coordinate(study)

[W 2022-09-27 16:44:57,502] Your study has only completed trials with missing parameters.


In [77]:
optuna.visualization.plot_param_importances(study)

In [78]:
optuna.visualization.plot_slice(study)